In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re

In [3]:
emotions_df = pd.read_csv("twitter.csv", 
                      encoding="latin", 
                      header=None, 
                      names=["target","id","date","query","username","content"]
                     )
emotions_df.head()

,target,id,date,query,username,content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
emotions_df.drop(["id","date","query","username"],axis=1, inplace=True)

In [5]:
emotions_df.target = emotions_df.target.replace({4: 1})

In [6]:
# Remove mentions
regex_mentions = r"@[A-Za-z0-9_]+"
# Remove links
regex_links = r"https?://[A-Za-z0-9./]+"
# Remove some special characters
regex_special = r"[^A-Za-z0-9]+"

In [12]:
# Remove mentions
emotions_df.content = emotions_df.content.apply(lambda x: re.sub(regex_mentions, " ", str(x).strip()))
# Remove links 
emotions_df.content = emotions_df.content.apply(lambda x: re.sub(regex_links, " ", str(x).strip()))
# Remove special characters
emotions_df

,target,content
0,0,Awww that s a bummer You shoulda got David Car...
1,0,is upset that he can t update his Facebook by ...
2,0,I dived many times for the ball Managed to sav...
3,0,my whole body feels itchy and like its on fire
4,0,no it s not behaving at all i m mad why am i h...
...,...,...
1599995,1,Just woke up Having no school is the best feel...
1599996,1,TheWDB com Very cool to hear old Walt intervie...
1599997,1,Are you ready for your MoJo Makeover Ask me fo...
1599998,1,Happy 38th Birthday to my boo of alll time Tup...


In [15]:
emotions_df

,target,content
0,0,Awww that s a bummer You shoulda got David Car...
1,0,is upset that he can t update his Facebook by ...
2,0,I dived many times for the ball Managed to sav...
3,0,my whole body feels itchy and like its on fire
4,0,no it s not behaving at all i m mad why am i h...
...,...,...
1599995,1,Just woke up Having no school is the best feel...
1599996,1,TheWDB com Very cool to hear old Walt intervie...
1599997,1,Are you ready for your MoJo Makeover Ask me fo...
1599998,1,Happy 38th Birthday to my boo of alll time Tup...


In [14]:
X = emotions_df['content']
y = emotions_df['target']

In [52]:
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=42)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(1280000,) (320000,) (1280000,) (320000,)


In [53]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=5000, stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [54]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=2, random_state=42)
xgb_model.fit(X_train_vec, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=2,
              num_parallel_tree=None, ...)

In [55]:
y_pred = xgb_model.predict(X_test_vec)

In [56]:
accuracy_score(y_test, y_pred)

0.732246875

In [57]:
import pickle
file = 'Trained1.sav'
pickle.dump(xgb_model, open(file, 'wb'))

In [58]:
#read model
model = pickle.load(open('Trained1.sav', 'rb'))

In [87]:
new = ["Our model doesn't work"]
second = ['I love this']
third = ['This is the best thing ever']
ori = ["i dont know i feel so lost"]

In [88]:
new_data = vectorizer.transform(new)

In [89]:
# #test model on data
#X_new = X_test_vec[30]

prediction = xgb_model.predict(new_data)
print(prediction)
if (prediction[0]== 0):
    print('negative tweet')
else:
    print('positive tweet')

[0]
negative tweet
